## Questão 02

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import random
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
def train(model, criterion, optimizer, train_loader, val_loader, num_epochs=100):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
        train_loss /= len(train_loader.dataset)
        train_losses.append(train_loss)

        model.eval()
        val_loss = 0.0
        for inputs, targets in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item() * inputs.size(0)
        val_loss /= len(val_loader.dataset)
        val_losses.append(val_loss)

        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

def test(model, criterion, val_loader):
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item() * inputs.size(0)
    test_loss /= len(val_loader.dataset)
    print(f'Test Loss: {test_loss:.4f}')
    return test_loss

### a) XOR

In [3]:
## Função de treinamento
def train_XOR(model, criterion, optimizer, train_loader, val_loader, epochs=100):
    train_losses = []
    val_losses = []
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels.unsqueeze(1))
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        train_loss = running_loss / len(train_loader)
        train_losses.append(train_loss)

        val_loss = evaluate_XOR(model, criterion, val_loader)
        val_losses.append(val_loss)
        
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

    return train_losses, val_losses


## Função de avaliação
def evaluate_XOR(model, criterion, val_loader):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels.unsqueeze(1))
            running_loss += loss.item()
    return running_loss / len(val_loader)

In [4]:
n = 10000
data_X = []
data_y = []

for i in range(n):
    instance = np.array([random.uniform(-5,5), random.uniform(-5,5)])

    if instance[0]*instance[1] > 0:
        label = 1
    elif instance[0]*instance[1] < 0:
        label = 0
    else:
        continue

    data_X.append(instance)
    data_y.append(label)


data_X = np.array(data_X)
data_y = torch.tensor(data_y, dtype=torch.float32)

O código abaixo realiza as seguintes tarefas:

- Divide os dados em conjuntos de treino, validação e teste usando `train_test_split`.
- Converte os dados para tensores do PyTorch.
- Define a arquitetura de uma rede neural simples com uma camada oculta usando a classe `PerceptronXOR`.
- Instancia o modelo, a função de perda (no caso, Binary Cross-Entropy Loss) e o otimizador (Adam).
- Define o `batch_size` e cria os DataLoader para os conjuntos de treino e validação.
- Treina o modelo usando a função `train`, que itera sobre os dados de treino e realiza o backpropagation.
- Avalia o modelo nos dados de teste usando a função `evaluate`.
- Calcula a acurácia do modelo nos dados de teste

In [5]:
x_train, x_test, y_train, y_test = train_test_split(data_X, data_y, test_size=0.2, random_state=10)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=10)

x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
x_val_tensor = torch.tensor(x_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

class PerceptronXOR(nn.Module):
    def __init__(self):
        super(PerceptronXOR, self).__init__()
        self.fc1 = nn.Linear(2, 4)  ## Camada de entrada
        self.fc2 = nn.Linear(4, 1)  ## Camada de saída

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

## Instanciar o modelo, função de perda e otimizador
model = PerceptronXOR()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

batch_size = 32
train_loader = DataLoader(TensorDataset(x_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(x_val_tensor, y_val_tensor), batch_size=batch_size)

train_losses, val_losses = train_XOR(model, criterion, optimizer, train_loader, val_loader, epochs=300)

test_loader = DataLoader(TensorDataset(x_test_tensor, y_test_tensor), batch_size=batch_size)
test_loss = evaluate_XOR(model, criterion, test_loader)
print(f"\nTest Loss: {test_loss:.4f}")

model.eval()
with torch.no_grad():
    outputs = model(x_test_tensor)
    predicted = torch.round(outputs)
    accuracy = (predicted == y_test_tensor.unsqueeze(1)).sum().item() / predicted.size(0)
    print(f"Acurácia: {accuracy:.2f}")


C:\Users\jahon\AppData\Local\Temp\ipykernel_11968\3227053614.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
C:\Users\jahon\AppData\Local\Temp\ipykernel_11968\3227053614.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_val_tensor = torch.tensor(y_val, dtype=torch.float32)
C:\Users\jahon\AppData\Local\Temp\ipykernel_11968\3227053614.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


Epoch 1/300, Train Loss: 0.6532, Val Loss: 0.6159
Epoch 2/300, Train Loss: 0.5587, Val Loss: 0.5115
Epoch 3/300, Train Loss: 0.4635, Val Loss: 0.4321
Epoch 4/300, Train Loss: 0.3970, Val Loss: 0.3768
Epoch 5/300, Train Loss: 0.3498, Val Loss: 0.3358
Epoch 6/300, Train Loss: 0.3136, Val Loss: 0.3036
Epoch 7/300, Train Loss: 0.2850, Val Loss: 0.2772
Epoch 8/300, Train Loss: 0.2622, Val Loss: 0.2555
Epoch 9/300, Train Loss: 0.2430, Val Loss: 0.2378
Epoch 10/300, Train Loss: 0.2263, Val Loss: 0.2206
Epoch 11/300, Train Loss: 0.2117, Val Loss: 0.2066
Epoch 12/300, Train Loss: 0.1989, Val Loss: 0.1947
Epoch 13/300, Train Loss: 0.1881, Val Loss: 0.1837
Epoch 14/300, Train Loss: 0.1783, Val Loss: 0.1743
Epoch 15/300, Train Loss: 0.1698, Val Loss: 0.1660
Epoch 16/300, Train Loss: 0.1618, Val Loss: 0.1584
Epoch 17/300, Train Loss: 0.1553, Val Loss: 0.1518
Epoch 18/300, Train Loss: 0.1490, Val Loss: 0.1456
Epoch 19/300, Train Loss: 0.1429, Val Loss: 0.1407
Epoch 20/300, Train Loss: 0.1378, Val Lo

In [6]:
predicted = predicted.numpy().flatten()
predicted

array([0., 0., 0., ..., 1., 0., 0.], dtype=float32)

In [7]:
## plot dos dados de teste em dois grupos de cores, de acordo com a variavel predict
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_test[predicted == 0, 0], y=x_test[predicted == 0, 1], mode='markers', name='0', marker=dict(color='blue')))
fig.add_trace(go.Scatter(x=x_test[predicted == 1, 0], y=x_test[predicted == 1, 1], mode='markers', name='1', marker=dict(color='red')))
fig.update_layout(title='Dados de teste', xaxis_title='X', yaxis_title='Y', height=600, width=900)
fig.show()

In [8]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(train_losses)), y=train_losses, mode='lines', name='Train Loss'))
fig.add_trace(go.Scatter(x=np.arange(len(val_losses)), y=val_losses, mode='lines', name='Val Loss'))
fig.update_layout(title='Train and Val Loss', xaxis_title='Epoch', yaxis_title='Loss', height=600, width=1200)
fig.add_trace(go.Scatter(x=[len(train_losses)], y=[test_loss], mode='markers', marker=dict(color='orange', size=10), name='Test Loss'))
fig.show()

### b) Log

In [9]:
def log10_data(n):
    x = np.linspace(1, 10, n).reshape(-1, 1)
    y = np.log10(x)
    return np.c_[x, y]

data = log10_data(10000)
x = np.array(data[:, 0], dtype=np.float32)
y = np.array(data[:, 1], dtype=np.float32)

In [10]:
# Convertendo para tensores
x_tensor = torch.from_numpy(x).unsqueeze(1)
y_tensor = torch.from_numpy(y).unsqueeze(1)

x_train, x_temp, y_train, y_temp = train_test_split(x_tensor, y_tensor, test_size=0.3)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.3)

batch_size = 64

# Criando DataLoader
train_data = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

val_data = TensorDataset(x_val, y_val)
val_loader = DataLoader(val_data, batch_size=batch_size)

# Definindo a arquitetura do modelo
class PerceptronLog10(nn.Module):
    def __init__(self):
        super(PerceptronLog10, self).__init__()
        self.fc1 = nn.Linear(1, 100)
        self.fc2 = nn.Linear(100, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = PerceptronLog10()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 50
train_losses = []
val_losses = []

train(model, criterion, optimizer, train_loader, val_loader, num_epochs)
test_loss = test(model, criterion, val_loader)

Epoch 1/50, Train Loss: 0.0514, Val Loss: 0.0034
Epoch 2/50, Train Loss: 0.0014, Val Loss: 0.0004
Epoch 3/50, Train Loss: 0.0003, Val Loss: 0.0001
Epoch 4/50, Train Loss: 0.0001, Val Loss: 0.0001
Epoch 5/50, Train Loss: 0.0000, Val Loss: 0.0000
Epoch 6/50, Train Loss: 0.0000, Val Loss: 0.0000
Epoch 7/50, Train Loss: 0.0000, Val Loss: 0.0000
Epoch 8/50, Train Loss: 0.0000, Val Loss: 0.0000
Epoch 9/50, Train Loss: 0.0000, Val Loss: 0.0000
Epoch 10/50, Train Loss: 0.0000, Val Loss: 0.0000
Epoch 11/50, Train Loss: 0.0000, Val Loss: 0.0000
Epoch 12/50, Train Loss: 0.0000, Val Loss: 0.0000
Epoch 13/50, Train Loss: 0.0000, Val Loss: 0.0000
Epoch 14/50, Train Loss: 0.0000, Val Loss: 0.0000
Epoch 15/50, Train Loss: 0.0000, Val Loss: 0.0000
Epoch 16/50, Train Loss: 0.0000, Val Loss: 0.0000
Epoch 17/50, Train Loss: 0.0000, Val Loss: 0.0000
Epoch 18/50, Train Loss: 0.0000, Val Loss: 0.0000
Epoch 19/50, Train Loss: 0.0000, Val Loss: 0.0000
Epoch 20/50, Train Loss: 0.0000, Val Loss: 0.0000
Epoch 21/

In [11]:
## valores preditos
predicted = model(x_test)
predicted = predicted.detach().numpy()

fig = go.Figure()
fig.add_trace(go.Scatter(x=x_test.flatten(), y=y_test.flatten(), mode='markers', name='Real', marker=dict(color='blue')))
fig.add_trace(go.Scatter(x=x_test.flatten(), y=predicted.flatten(), mode='markers', name='Predicted', marker=dict(color='red')))
fig.update_layout(title='Dados de teste', xaxis_title='X', yaxis_title='Y', height=600, width=900)
fig.show()

In [12]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(train_losses)), y=train_losses, mode='lines', name='Train Loss'))
fig.add_trace(go.Scatter(x=np.arange(len(val_losses)), y=val_losses, mode='lines', name='Val Loss'))
fig.update_layout(title='Train and Val Loss', xaxis_title='Epoch', yaxis_title='Loss', height=600, width=1200)
fig.add_trace(go.Scatter(x=[len(train_losses)], y=[test_loss], mode='markers', marker=dict(color='orange', size=10), name='Test Loss'))
fig.show()

### c) Polinomial

In [13]:
def f(x):
    return 10*x**5 + 5*x**4 + 2*x**3 - 0.5*x**2 + 3*x + 2

def fx_data(n):
    x = np.linspace(0, 5, n).reshape(-1, 1)
    y = f(x)
    return np.c_[x, y]

data = fx_data(10000)
x = np.array(data[:, 0], dtype=np.float32)
y = np.array(data[:, 1], dtype=np.float32)

In [14]:
# Convertendo para tensores
x_tensor = torch.from_numpy(x).unsqueeze(1)
y_tensor = torch.from_numpy(y).unsqueeze(1)

x_train, x_temp, y_train, y_temp = train_test_split(x_tensor, y_tensor, test_size=0.3)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.3)

batch_size = 64

# Criando DataLoader
train_data = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

val_data = TensorDataset(x_val, y_val)
val_loader = DataLoader(val_data, batch_size=batch_size)

# Definindo a arquitetura do modelo
class PerceptronPol(nn.Module):
    def __init__(self):
        super(PerceptronPol, self).__init__()
        self.fc1 = nn.Linear(1, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 100)
        self.fc4 = nn.Linear(100, 100)
        self.fc5 = nn.Linear(100, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc5(x)
        return x

model = PerceptronPol()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 50
train_losses = []
val_losses = []

train(model, criterion, optimizer, train_loader, val_loader, num_epochs)
test_loss = test(model, criterion, val_loader)

Epoch 1/50, Train Loss: 95419337.0674, Val Loss: 36072329.9048
Epoch 2/50, Train Loss: 35807908.9531, Val Loss: 28091592.3695
Epoch 3/50, Train Loss: 24125603.1726, Val Loss: 14587310.4038
Epoch 4/50, Train Loss: 9836754.5966, Val Loss: 4846055.6514
Epoch 5/50, Train Loss: 3132839.3409, Val Loss: 1893956.2786
Epoch 6/50, Train Loss: 1844211.4593, Val Loss: 1526359.0412
Epoch 7/50, Train Loss: 1403318.9719, Val Loss: 936793.2521
Epoch 8/50, Train Loss: 796124.4042, Val Loss: 525712.0720
Epoch 9/50, Train Loss: 457004.0928, Val Loss: 280139.0149
Epoch 10/50, Train Loss: 287220.8241, Val Loss: 164684.1166
Epoch 11/50, Train Loss: 154612.1264, Val Loss: 91650.1381
Epoch 12/50, Train Loss: 92531.1810, Val Loss: 60309.9524
Epoch 13/50, Train Loss: 63883.2172, Val Loss: 42556.5568
Epoch 14/50, Train Loss: 44008.1591, Val Loss: 34757.3537
Epoch 15/50, Train Loss: 35936.1637, Val Loss: 22763.5678
Epoch 16/50, Train Loss: 25136.5228, Val Loss: 28277.0980
Epoch 17/50, Train Loss: 19430.1434, Val 

In [15]:
## valores preditos
predicted = model(x_test)
predicted = predicted.detach().numpy()

fig = go.Figure()
fig.add_trace(go.Scatter(x=x_test.flatten(), y=y_test.flatten(), mode='markers', name='Real', marker=dict(color='blue')))
fig.add_trace(go.Scatter(x=x_test.flatten(), y=predicted.flatten(), mode='markers', name='Predicted', marker=dict(color='red')))
fig.update_layout(title='Dados de teste', xaxis_title='X', yaxis_title='Y', height=600, width=900)
fig.show()

In [16]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(train_losses)), y=train_losses, mode='lines', name='Train Loss'))
fig.add_trace(go.Scatter(x=np.arange(len(val_losses)), y=val_losses, mode='lines', name='Val Loss'))
fig.update_layout(title='Train and Val Loss', xaxis_title='Epoch', yaxis_title='Loss', height=600, width=1200)
fig.add_trace(go.Scatter(x=[len(train_losses)], y=[test_loss], mode='markers', marker=dict(color='orange', size=10), name='Test Loss'))
fig.show()